In [1]:
import models

2023-11-17 14:12:42.299608: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow as tf

In [3]:
m = models.iunet2(input_size=(96, 96, 1), dropout_rate=0.5, batch_norm=False, n_classes=3)
m.summary()

2023-11-17 14:12:45.684712: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 14:12:46.337501: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11551 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:8a:00.0, compute capability: 3.7


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 1)]  0           []                               
                                                                                                  
 switch_normalization (SwitchNo  (None, 96, 96, 1)   10          ['input_1[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 96, 96, 1)    0           ['switch_normalization[0][

In [13]:
from training_cmdline import *

2023-11-17 14:13:46.891072: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11551 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:8a:00.0, compute capability: 3.7


In [19]:
from tensorflow.keras.optimizers.experimental import SGD
from functools import partial

In [15]:
optimizer = SGD(learning_rate = 0.0025, weight_decay = 1e-7, momentum = 0.9, nesterov = True)

In [20]:
dicelossw = partial(diceloss, weights = [1, 3, 1])

In [21]:
m.compile(loss=dicelossw, optimizer=optimizer, metrics=['accuracy'], jit_compile=False)

In [31]:
import os
J_ROOT = os.readlink('/proc/%s/cwd' % os.environ['JPY_PARENT_PID'])
npzpath = os.path.join(J_ROOT, 'data', 'pecot_dice', "annotated_datasets/23-09-15_Annotation/23-11-15_32x32_cNp_erdi_nSeg/npz/data.npz")

In [32]:
train_dict, (X_test, Y_test) = get_data_sample(npzpath)

In [33]:
train_generator = random_sample_generator(
        train_dict["channels"], train_dict["batch"], train_dict["pixels_x"], 
        train_dict["pixels_y"], train_dict["labels"], batch_size, n_channels, 
        n_classes, train_dict["win_x"], train_dict["win_y"], augmentation
)

NameError: name 'batch_size' is not defined

In [ ]:
# fit the model on the batches generated by datagen.flow()
loss_history = model.fit(
    train_generator, steps_per_epoch = int(len(train_dict["batch"])/batch_size),
    epochs = n_epoch, validation_data = (X_test,Y_test),
    callbacks = [
        ModelCheckpoint(
            file_name_save, monitor='val_loss', verbose=0, save_best_only=True, 
            mode='auto', save_weights_only=True
        ), LearningRateScheduler(lr_sched), tensorboard_callback,
        EarlyStopping(
            monitor='val_loss', mode='min', min_delta=0.001,
            verbose=1, patience=15
        )
    ]
)

In [5]:
input = tf.random.uniform((1, 96, 96, 1))

In [6]:
pool = tf.constant((2, 2))

In [7]:
stride = tf.constant((1, 1,))

In [8]:
from tensorflow.keras.layers import MaxPooling2D

In [9]:
mpd = MaxPooling2D(pool_size=(16, 16))

In [10]:
mpd(input).shape

2023-11-17 14:12:49.652910: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8201


TensorShape([1, 6, 6, 1])

In [11]:
mpd(input).shape

TensorShape([1, 6, 6, 1])